In [1]:
# cloud_cost_optimizer.py
# Cloud Cost Analyzer & Optimizer - Standalone Version
# No external dependencies required

import json
import logging
from typing import Dict, List, Optional, Tuple
from dataclasses import dataclass, asdict
from datetime import datetime, timedelta
from enum import Enum
import random
from collections import defaultdict

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class Priority(Enum):
    CRITICAL = "critical"
    HIGH = "high"
    MEDIUM = "medium"
    LOW = "low"

@dataclass
class CostData:
    service: str
    amount: float
    currency: str
    start_date: datetime
    end_date: datetime
    tags: Dict[str, str]

@dataclass
class Recommendation:
    resource_type: str
    resource_id: str
    current_cost_monthly: float
    potential_savings_monthly: float
    recommendation_text: str
    priority: str
    implementation_effort: str
    estimated_implementation_time_hours: int

@dataclass
class CostTrend:
    service: str
    dates: List[str]
    costs: List[float]
    average_cost: float
    trend: str  # 'increasing', 'decreasing', 'stable'

@dataclass
class BudgetAlert:
    budget_name: str
    limit_monthly: float
    actual_spent: float
    forecasted_monthly: float
    percentage_used: float
    status: str  # 'on_track', 'warning', 'exceeded'

class MockCostData:
    """Generate mock cost data for standalone operation"""

    @staticmethod
    def generate_service_costs(num_days: int = 30) -> List[CostData]:
        """Generate mock AWS service costs"""
        services = [
            'Amazon EC2', 'Amazon S3', 'Amazon RDS',
            'AWS Lambda', 'Amazon DynamoDB', 'Amazon CloudFront',
            'AWS Data Transfer', 'Amazon SQS', 'Amazon SNS',
            'AWS Glue', 'Amazon Athena', 'AWS KMS'
        ]

        costs = []
        base_date = datetime.now() - timedelta(days=num_days)

        for day in range(num_days):
            current_date = base_date + timedelta(days=day)

            for service in services:
                amount = random.uniform(10, 500)

                costs.append(CostData(
                    service=service,
                    amount=amount,
                    currency='USD',
                    start_date=current_date,
                    end_date=current_date + timedelta(days=1),
                    tags={'Environment': random.choice(['prod', 'staging', 'dev'])}
                ))

        return costs

    @staticmethod
    def generate_underutilized_resources() -> Dict[str, List[Dict]]:
        """Generate mock underutilized resources"""
        return {
            'ec2_instances': [
                {
                    'id': f'i-{random.randint(100000000000, 999999999999):012x}',
                    'type': random.choice(['t2.small', 't2.medium', 't3.large']),
                    'cpu_utilization': random.uniform(1, 15),
                    'monthly_cost': random.uniform(20, 150),
                    'status': 'running'
                }
                for _ in range(5)
            ],
            'rds_instances': [
                {
                    'id': f'db-{random.randint(1000, 9999)}',
                    'class': random.choice(['db.t3.micro', 'db.t3.small', 'db.m5.large']),
                    'connections': random.uniform(0, 10),
                    'monthly_cost': random.uniform(50, 500),
                    'status': 'available'
                }
                for _ in range(3)
            ],
            'ebs_volumes': [
                {
                    'id': f'vol-{random.randint(100000, 999999)}',
                    'size_gb': random.randint(50, 500),
                    'type': random.choice(['gp2', 'gp3', 'io1']),
                    'iops': random.randint(100, 20000),
                    'monthly_cost': random.uniform(5, 100),
                    'status': 'available'
                }
                for _ in range(4)
            ]
        }

class CostAnalyzer:
    """Analyzes cloud costs"""

    def __init__(self):
        self.cost_data = MockCostData.generate_service_costs()
        self.underutilized_resources = MockCostData.generate_underutilized_resources()
        logger.info("Cost Analyzer initialized with mock data")

    def calculate_total_cost(self, start_date: Optional[datetime] = None,
                           end_date: Optional[datetime] = None) -> float:
        """Calculate total cost for period"""
        total = 0.0

        for cost in self.cost_data:
            if start_date and cost.start_date < start_date:
                continue
            if end_date and cost.end_date > end_date:
                continue
            total += cost.amount

        return total

    def get_cost_by_service(self) -> Dict[str, float]:
        """Get costs grouped by service"""
        service_costs = defaultdict(float)

        for cost in self.cost_data:
            service_costs[cost.service] += cost.amount

        return dict(service_costs)

    def get_cost_by_tag(self, tag_key: str) -> Dict[str, float]:
        """Get costs grouped by tag"""
        tag_costs = defaultdict(float)

        for cost in self.cost_data:
            if tag_key in cost.tags:
                tag_value = cost.tags[tag_key]
                tag_costs[tag_value] += cost.amount

        return dict(tag_costs)

    def get_cost_trend(self, service: str, days: int = 30) -> Optional[CostTrend]:
        """Get cost trend for a service"""
        daily_costs = defaultdict(float)

        for cost in self.cost_data:
            if cost.service == service:
                date_str = cost.start_date.strftime('%Y-%m-%d')
                daily_costs[date_str] += cost.amount

        if not daily_costs:
            return None

        dates = sorted(daily_costs.keys())
        costs = [daily_costs[date] for date in dates]
        average = sum(costs) / len(costs) if costs else 0

        # Determine trend
        if len(costs) >= 2:
            first_half_avg = sum(costs[:len(costs)//2]) / (len(costs)//2)
            second_half_avg = sum(costs[len(costs)//2:]) / (len(costs) - len(costs)//2)

            if second_half_avg > first_half_avg * 1.1:
                trend = 'increasing'
            elif second_half_avg < first_half_avg * 0.9:
                trend = 'decreasing'
            else:
                trend = 'stable'
        else:
            trend = 'stable'

        return CostTrend(
            service=service,
            dates=dates,
            costs=costs,
            average_cost=average,
            trend=trend
        )

    def forecast_monthly_cost(self, days_of_data: int = 30) -> Tuple[float, float]:
        """Forecast monthly cost based on current data"""
        if not self.cost_data:
            return 0.0, 0.0

        # Calculate daily average
        daily_average = self.calculate_total_cost() / days_of_data

        # Project to 30 days
        monthly_forecast = daily_average * 30

        # Calculate standard deviation for confidence
        costs_per_day = defaultdict(float)
        for cost in self.cost_data:
            date = cost.start_date.strftime('%Y-%m-%d')
            costs_per_day[date] += cost.amount

        daily_costs = list(costs_per_day.values())
        if len(daily_costs) > 1:
            mean = sum(daily_costs) / len(daily_costs)
            variance = sum((x - mean) ** 2 for x in daily_costs) / len(daily_costs)
            std_dev = variance ** 0.5
        else:
            std_dev = 0.0

        return monthly_forecast, std_dev

class CostOptimizer:
    """Optimizes cloud costs"""

    def __init__(self, analyzer: CostAnalyzer):
        self.analyzer = analyzer
        self.recommendations: List[Recommendation] = []

    def analyze_underutilized_ec2(self) -> List[Recommendation]:
        """Analyze underutilized EC2 instances"""
        recommendations = []

        for instance in self.analyzer.underutilized_resources['ec2_instances']:
            cpu = instance['cpu_utilization']

            if cpu < 10:
                priority = Priority.HIGH.value
                savings = instance['monthly_cost'] * 0.8
            elif cpu < 20:
                priority = Priority.MEDIUM.value
                savings = instance['monthly_cost'] * 0.5
            else:
                continue

            rec = Recommendation(
                resource_type='EC2 Instance',
                resource_id=instance['id'],
                current_cost_monthly=instance['monthly_cost'],
                potential_savings_monthly=savings,
                recommendation_text=f"Instance {instance['id']} ({instance['type']}) has low CPU utilization ({cpu:.1f}%). Consider downsizing or terminating.",
                priority=priority,
                implementation_effort='Low',
                estimated_implementation_time_hours=1
            )
            recommendations.append(rec)

        return recommendations

    def analyze_unattached_volumes(self) -> List[Recommendation]:
        """Analyze unattached EBS volumes"""
        recommendations = []

        for volume in self.analyzer.underutilized_resources['ebs_volumes']:
            rec = Recommendation(
                resource_type='EBS Volume',
                resource_id=volume['id'],
                current_cost_monthly=volume['monthly_cost'],
                potential_savings_monthly=volume['monthly_cost'],
                recommendation_text=f"Unattached EBS volume {volume['id']} ({volume['size_gb']}GB). Delete if not needed.",
                priority=Priority.MEDIUM.value,
                implementation_effort='Low',
                estimated_implementation_time_hours=0.5
            )
            recommendations.append(rec)

        return recommendations

    def analyze_idle_databases(self) -> List[Recommendation]:
        """Analyze idle RDS instances"""
        recommendations = []

        for db in self.analyzer.underutilized_resources['rds_instances']:
            connections = db['connections']

            if connections < 5:
                priority = Priority.HIGH.value
                savings = db['monthly_cost'] * 0.7
            elif connections < 15:
                priority = Priority.MEDIUM.value
                savings = db['monthly_cost'] * 0.4
            else:
                continue

            rec = Recommendation(
                resource_type='RDS Database',
                resource_id=db['id'],
                current_cost_monthly=db['monthly_cost'],
                potential_savings_monthly=savings,
                recommendation_text=f"RDS instance {db['id']} ({db['class']}) has low connections ({connections:.0f}). Consider stopping or downsizing.",
                priority=priority,
                implementation_effort='Medium',
                estimated_implementation_time_hours=2
            )
            recommendations.append(rec)

        return recommendations

    def analyze_data_transfer_costs(self) -> List[Recommendation]:
        """Analyze data transfer costs"""
        recommendations = []

        service_costs = self.analyzer.get_cost_by_service()

        if 'AWS Data Transfer' in service_costs:
            transfer_cost = service_costs['AWS Data Transfer']

            if transfer_cost > 100:
                rec = Recommendation(
                    resource_type='Data Transfer',
                    resource_id='aws-data-transfer',
                    current_cost_monthly=transfer_cost,
                    potential_savings_monthly=transfer_cost * 0.3,
                    recommendation_text=f"High data transfer costs (${transfer_cost:.2f}/month). Consider using CloudFront CDN or VPC endpoints.",
                    priority=Priority.MEDIUM.value,
                    implementation_effort='High',
                    estimated_implementation_time_hours=8
                )
                recommendations.append(rec)

        return recommendations

    def generate_optimization_report(self) -> Dict:
        """Generate comprehensive optimization report"""
        self.recommendations = []

        # Run all analyses
        self.recommendations.extend(self.analyze_underutilized_ec2())
        self.recommendations.extend(self.analyze_unattached_volumes())
        self.recommendations.extend(self.analyze_idle_databases())
        self.recommendations.extend(self.analyze_data_transfer_costs())

        # Calculate totals
        total_savings = sum(r.potential_savings_monthly for r in self.recommendations)
        total_current_cost = sum(r.current_cost_monthly for r in self.recommendations)

        # Analyze costs
        service_costs = self.analyzer.get_cost_by_service()
        top_services = sorted(service_costs.items(), key=lambda x: x[1], reverse=True)[:5]

        # Get forecast
        monthly_forecast, std_dev = self.analyzer.forecast_monthly_cost()

        # Group by priority
        by_priority = defaultdict(list)
        for rec in self.recommendations:
            by_priority[rec.priority].append(rec)

        report = {
            'summary': {
                'total_recommendations': len(self.recommendations),
                'total_potential_monthly_savings': round(total_savings, 2),
                'total_annual_savings': round(total_savings * 12, 2),
                'current_monthly_cost': round(total_current_cost, 2),
                'forecasted_monthly_cost': round(monthly_forecast, 2)
            },
            'recommendations_by_priority': {
                priority: len(recs)
                for priority, recs in by_priority.items()
            },
            'top_services': [
                {'service': svc, 'monthly_cost': round(cost, 2)}
                for svc, cost in top_services
            ],
            'detailed_recommendations': [
                {
                    'id': f"rec-{i+1}",
                    'resource_type': r.resource_type,
                    'resource_id': r.resource_id,
                    'current_cost_monthly': round(r.current_cost_monthly, 2),
                    'potential_savings_monthly': round(r.potential_savings_monthly, 2),
                    'recommendation': r.recommendation_text,
                    'priority': r.priority,
                    'effort': r.implementation_effort,
                    'implementation_hours': r.estimated_implementation_time_hours
                }
                for i, r in enumerate(sorted(self.recommendations,
                                            key=lambda x: x.potential_savings_monthly,
                                            reverse=True))
            ]
        }

        return report

    def create_budget_alerts(self, monthly_budget: float = 10000.0) -> List[BudgetAlert]:
        """Create budget alerts"""
        service_costs = self.analyzer.get_cost_by_service()
        total_cost = self.analyzer.calculate_total_cost()

        monthly_forecast, _ = self.analyzer.forecast_monthly_cost()

        # Determine status
        if total_cost > monthly_budget:
            status = 'exceeded'
        elif total_cost > monthly_budget * 0.8:
            status = 'warning'
        else:
            status = 'on_track'

        alerts = [
            BudgetAlert(
                budget_name='Overall Monthly Budget',
                limit_monthly=monthly_budget,
                actual_spent=total_cost,
                forecasted_monthly=monthly_forecast,
                percentage_used=(total_cost / monthly_budget) * 100 if monthly_budget > 0 else 0,
                status=status
            )
        ]

        # Create service-level budgets (10% of total budget per service)
        service_budget = monthly_budget / 10

        for service, cost in sorted(service_costs.items(), key=lambda x: x[1], reverse=True)[:5]:
            if cost > service_budget:
                svc_status = 'warning'
            else:
                svc_status = 'on_track'

            alerts.append(BudgetAlert(
                budget_name=f'{service} Budget',
                limit_monthly=service_budget,
                actual_spent=cost,
                forecasted_monthly=cost * (30 / len(self.analyzer.cost_data)),
                percentage_used=(cost / service_budget) * 100 if service_budget > 0 else 0,
                status=svc_status
            ))

        return alerts

def main():
    """Main execution function"""
    print("=" * 70)
    print(" Cloud Cost Analyzer & Optimizer - Standalone Demo")
    print("=" * 70)
    print()

    # Initialize analyzer
    analyzer = CostAnalyzer()

    # Get basic metrics
    print("💰 Cost Summary (Last 30 Days):")
    total_cost = analyzer.calculate_total_cost()
    print(f"  Total Cost: ${total_cost:.2f}")
    print()

    # Show costs by service
    print("=" * 70)
    print(" Cost by Service (Top 10)")
    print("=" * 70)
    service_costs = analyzer.get_cost_by_service()
    for service, cost in sorted(service_costs.items(), key=lambda x: x[1], reverse=True)[:10]:
        percentage = (cost / total_cost * 100) if total_cost > 0 else 0
        print(f"  {service}: ${cost:.2f} ({percentage:.1f}%)")
    print()

    # Show costs by environment
    print("=" * 70)
    print(" Cost by Environment")
    print("=" * 70)
    env_costs = analyzer.get_cost_by_tag('Environment')
    for env, cost in sorted(env_costs.items()):
        percentage = (cost / total_cost * 100) if total_cost > 0 else 0
        print(f"  {env}: ${cost:.2f} ({percentage:.1f}%)")
    print()

    # Show forecast
    print("=" * 70)
    print(" Cost Forecast")
    print("=" * 70)
    monthly_forecast, std_dev = analyzer.forecast_monthly_cost()
    print(f"  Forecasted Monthly Cost: ${monthly_forecast:.2f}")
    print(f"  Standard Deviation: ${std_dev:.2f}")
    print(f"  Forecasted Annual Cost: ${monthly_forecast * 12:.2f}")
    print()

    # Run optimizer
    print("=" * 70)
    print(" Generating Optimization Recommendations")
    print("=" * 70)
    print()

    optimizer = CostOptimizer(analyzer)
    report = optimizer.generate_optimization_report()

    print(f"Total Recommendations: {report['summary']['total_recommendations']}")
    print(f"Potential Monthly Savings: ${report['summary']['total_potential_monthly_savings']:.2f}")
    print(f"Potential Annual Savings: ${report['summary']['total_annual_savings']:.2f}")
    print()

    # Show recommendations by priority
    print("Recommendations by Priority:")
    for priority, count in sorted(report['recommendations_by_priority'].items()):
        print(f"  {priority}: {count}")
    print()

    # Show top recommendations
    print("=" * 70)
    print(" Top 5 Cost Saving Recommendations")
    print("=" * 70)
    for rec in report['detailed_recommendations'][:5]:
        print(f"\n{rec['id']}: {rec['resource_type']} - {rec['resource_id']}")
        print(f"  Priority: {rec['priority'].upper()}")
        print(f"  Savings: ${rec['potential_savings_monthly']:.2f}/month")
        print(f"  Recommendation: {rec['recommendation']}")
        print(f"  Effort: {rec['effort']} ({rec['implementation_hours']} hours)")
    print()

    # Generate budget alerts
    print("=" * 70)
    print(" Budget Alerts")
    print("=" * 70)
    alerts = optimizer.create_budget_alerts(monthly_budget=5000.0)

    for alert in alerts[:5]:
        status_emoji = "🔴" if alert.status == 'exceeded' else "🟡" if alert.status == 'warning' else "🟢"
        print(f"\n{status_emoji} {alert.budget_name}")
        print(f"  Budget: ${alert.limit_monthly:.2f}")
        print(f"  Spent: ${alert.actual_spent:.2f}")
        print(f"  Usage: {alert.percentage_used:.1f}%")
        print(f"  Status: {alert.status.upper()}")
    print()

    # Export report
    print("=" * 70)
    print(" Exporting Reports")
    print("=" * 70)

    with open('cost_optimization_report.json', 'w') as f:
        json.dump(report, f, indent=2)
    print("✓ Cost optimization report saved to: cost_optimization_report.json")

    # Export budget alerts
    alerts_data = [
        {
            'budget_name': a.budget_name,
            'limit_monthly': a.limit_monthly,
            'actual_spent': a.actual_spent,
            'forecasted_monthly': a.forecasted_monthly,
            'percentage_used': a.percentage_used,
            'status': a.status
        }
        for a in alerts
    ]

    with open('budget_alerts.json', 'w') as f:
        json.dump(alerts_data, f, indent=2)
    print("✓ Budget alerts saved to: budget_alerts.json")

    print()
    print("=" * 70)
    print(" Demo completed successfully!")
    print("=" * 70)

if __name__ == "__main__":
    main()

 Cloud Cost Analyzer & Optimizer - Standalone Demo

💰 Cost Summary (Last 30 Days):
  Total Cost: $92193.87

 Cost by Service (Top 10)
  AWS Glue: $9335.38 (10.1%)
  AWS Lambda: $8811.02 (9.6%)
  Amazon EC2: $8686.25 (9.4%)
  Amazon Athena: $8414.58 (9.1%)
  Amazon S3: $7791.19 (8.5%)
  Amazon DynamoDB: $7405.33 (8.0%)
  Amazon SNS: $7310.71 (7.9%)
  Amazon RDS: $7278.22 (7.9%)
  Amazon CloudFront: $7083.35 (7.7%)
  AWS KMS: $7044.50 (7.6%)

 Cost by Environment
  dev: $24660.06 (26.7%)
  prod: $32754.30 (35.5%)
  staging: $34779.50 (37.7%)

 Cost Forecast
  Forecasted Monthly Cost: $92193.87
  Standard Deviation: $368.96
  Forecasted Annual Cost: $1106326.49

 Generating Optimization Recommendations

Total Recommendations: 13
Potential Monthly Savings: $3016.58
Potential Annual Savings: $36198.90

Recommendations by Priority:
  high: 3
  medium: 10

 Top 5 Cost Saving Recommendations

rec-1: Data Transfer - aws-data-transfer
  Priority: MEDIUM
  Savings: $1969.06/month
  Recommendation